In [1]:
import glob
import os

In [2]:
CLIP_PATH = "data/Patient_1/"

def get_clips(data_folder):

    # Get all clips
    clips = os.listdir(data_folder)

    # Preictal recordings - time-series segments of the measurement before a seizure occura
    clips_preictal = glob.glob(os.path.join(data_folder, "*preictal*"))

    # Interictial segments - segments with no oncoming seizures
    clips_interictial = glob.glob(os.path.join(data_folder, "*interictal*"))

    return clips_interictial, clips_preictal


# Get EEG recordings
clips_interictal, clips_preictal = get_clips(data_folder = CLIP_PATH)
clips_interictal[0:2]

['data/Patient_1/Patient_1_interictal_segment_0020.mat',
 'data/Patient_1/Patient_1_interictal_segment_0042.mat']

In [3]:
import os
import numpy as np
from scipy.io import loadmat

# Construct LSTM sequences from one segment
def lstm_sequence(input_segment, target, sampling_freq, window, stride, block_s = 60):
    """ Function for generating blocks of LSTM input tensors
        input_segment : The EEG segment
        target        : 1/0 (preictal/interictial); None for test
        sampling_freq : Samplig frequency
        window        : Window size for 1d convolutions on each block
        stride        : Stride size of the 1d convolution
        block_s       : Size of the block in seconds (default = 60)
    """

    # Dimensions
    n_channels, T_segment = input_segment.shape

    # Determine block dimensions
    block_len = sampling_freq * block_s   # Length of each block
    n_blocks = (T_segment-1) // block_len # Number of blocks
    blocks = [block for block in range(0,(n_blocks+1)*block_len,block_len)]

    # Determine the sequence length for LSTM
    div = (block_len - window)%stride
    if (div != 0):
        pad = stride - div # Size of padding neded
    else:
        pad = 0

    seq_len = (block_len + pad - window) // stride

    # Initiate tensor
    X = np.zeros((n_blocks, seq_len, n_channels))

    # Loop over blocks and fill X
    for ib in range(n_blocks):
        # Get block
        data_block = input_segment[:, blocks[ib]:blocks[ib+1]]

        # Pad if necessary
        if (pad !=0):
            data_block = np.concatenate((data_block, np.zeros((n_channels, pad))), axis=1)

        # 1d convolution by mean
        index = 0
        for j in range(seq_len):
            X[ib, j, :] = np.mean(data_block[:, (index+j):(index+j+seq_len)], axis = 1)

    # Fill in the target
    if (target == 1):
        Y = np.ones(n_blocks)
    elif(target == 0):
        Y = np.zeros(n_blocks)
    else:
        Y = None

    return X, Y, n_blocks

# Collect all the segments to build a tesnsor input for LSTM
def lstm_build_input(clips, target, window, stride, block_s = 60):
    """ Collect all the data and build sequences for LSTM
        clips              : List of clips
        target             : 1/0 (preictal/interictial); None for test set
        window             : Window size for 1d convolutions
        stride             : Length of the stride in 1d convolution
        block_s            : Size of the block in seconds (default = 60)
    """

    # Number of clips
    n_clips = len(clips)

    # Loop over all clips and store data
    iclip = 0
    for file in clips:
        clip = loadmat(file)
        segment_name = list(clip.keys())[3] # Get segment name
        input_segment = clip[segment_name][0][0][0] # Get electrode data
        sampling_freq = np.squeeze(clip[segment_name][0][0][2]) # Sampling frequency

        # Get number of channels
        n_channels = clip[segment_name][0][0][0].shape[0]

        # Get tensor input and targets from blocks
        X, Y, n_blocks = lstm_sequence(input_segment, target, sampling_freq, window, stride, block_s)

        # Concatenate the tensor and target vector
        if (iclip == 0):
            X_train = X
            Y_train = Y[:,None] if Y is not None else None
        else:
            X_train = np.vstack((X_train,X))
            Y_train = np.vstack((Y_train,Y[:,None])) if Y is not None else None

        iclip +=1

    return X_train, Y_train


# Window, stride and block_s
window = 16000
stride = 100
block_s = 60

X_1, Y_1 = lstm_build_input(clips_preictal, 1, window, stride)
X_0, Y_0 = lstm_build_input(clips_interictal, 0, window, stride)

# Scale the data
X_1 = X_1 / np.max(np.abs(X_1), axis=1)[:,None,:]
X_0 = X_0 / np.max(np.abs(X_0), axis=1)[:,None,:]

# Combine the data
X = np.concatenate((X_0, X_1), axis = 0)
Y = np.concatenate((Y_0, Y_1), axis = 0)
Y = np.squeeze(Y)

print("Data shape = ", X.shape)

Data shape =  (612, 2840, 15)


In [36]:
Y.shape

(612,)

In [37]:
# Normalize
X = X / np.max(np.abs(X), axis=1)[:,None,:]

# Shuffle
np.random.seed(1)
shuffle = np.random.choice(np.arange(len(Y)), size=len(Y), replace=False)
X = X[shuffle]
Y = Y[shuffle]

In [38]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
model.add(layers.Input(shape=(2840, 15)))
model.add(layers.LSTM(64))
model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))

In [39]:
import numpy
from keras.callbacks import Callback

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


class RocCallback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_train = model.predict(self.x)
        roc_train = roc_auc_score(self.y, y_pred_train)
        y_pred_val = model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('roc-auc_train: ', roc_train)
        print('roc-auc_val: ', roc_val)
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

n_channels = 15
# Function that trains the model
def train_model(X, Y):
    X = X.reshape(X.shape[0], -1, n_channels)   # reshape DH table to 3d numpy array
    X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, stratify=Y, test_size = 0.1)
    roc = RocCallback(training_data=(X_train, Y_train), validation_data=(X_valid, Y_valid))
    model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
    # model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), callbacks=[roc], batch_size = 200, epochs=100)
    model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), callbacks=[roc], batch_size = 200, epochs=100)

# Function that gets the model's predictions on input data
def predict_with_model(X):
    X = X.reshape(X.shape[0], -1, n_channels)  # reshape DH table to 3d numpy array
    Y_pred = model.predict(X, batch_size=200)
    return Y_pred

# Function to extract a list element at a given index
def get_predicted_class(data, idx):
    return data[idx]

# Split the data into training and test datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size = 0.2)

# Convert numpy arrays X_train and Y_train to DH table X_table
n_rows = X_train.shape[0]
n_cols = X_train.shape[1] * X_train.shape[2]
column_names = ['Col_'+str(i) for i in range(n_cols)]
X_reshaped = X_train.reshape(n_rows, n_cols)
# X_table = numpy.to_table(X_reshaped, cols=column_names)

def add_class_col(index):
    y_class = [int(i) for i in Y_train.tolist()]
    return y_class[index]

# X_table = X_table.update(["Class = (int)add_class_col(i)"])


train_model(X_train, Y_train)

# # Train the model
# learn.learn(
#     table=X_table,
#     model_func=train_model,
#     inputs=[learn.Input(column_names, table_to_array_double), learn.Input(["Class"], table_to_array_int)],
#     outputs=None,
#     batch_size=200
# )


# Convert numpy array X_test to DH table X_table_test
X_reshaped_test = X_test.reshape(X_test.shape[0], n_cols)
# X_table_test = numpy.to_table(X_reshaped_test, cols=column_names)

# Use the learn function to create a new table that contains predicted values
y_pred = predict_with_model(X_test)

# predicted = learn.learn(
#     table=X_table_test,
#     model_func=predict_with_model,
#     inputs=[learn.Input(column_names, table_to_array_double)],
#     outputs=[learn.Output("PredictedClass", get_predicted_class, "int")],
#     batch_size=200
# )


Epoch 1/100
2/2 [==============================] - 0s 116ms/step
roc-auc_train:  0.5429438058748404
roc-auc_val:  0.6923076923076924
3/3 [==============================] - 7s 2s/step - loss: 0.8594 - accuracy: 0.5114 - val_loss: 0.6702 - val_accuracy: 0.6939
Epoch 2/100
2/2 [==============================] - 0s 110ms/step
roc-auc_train:  0.5883354618986802
roc-auc_val:  0.6623931623931625
3/3 [==============================] - 5s 2s/step - loss: 0.7480 - accuracy: 0.5159 - val_loss: 0.6743 - val_accuracy: 0.6735
Epoch 3/100
2/2 [==============================] - 0s 120ms/step
roc-auc_train:  0.6478554704129417
roc-auc_val:  0.6111111111111112
3/3 [==============================] - 5s 2s/step - loss: 0.6858 - accuracy: 0.5591 - val_loss: 0.6798 - val_accuracy: 0.6327
Epoch 4/100
2/2 [==============================] - 0s 111ms/step
roc-auc_train:  0.7002979991485738
roc-auc_val:  0.5555555555555556
3/3 [==============================] - 5s 2s/step - loss: 0.6525 - accuracy: 0.6000 - val_

In [4]:
y_pred = predict_with_model(X_test)
y_test = Y_test

from sklearn.metrics import precision_score, recall_score
y_pred_c = y_pred.round().astype(int)
precision = precision_score(y_test, y_pred_c)
recall = recall_score(y_test, y_pred_c)

loss, accuracy = model.evaluate(X_test, y_test)

print('Loss:', loss)
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
Y_test.sum(), y_pred_c.sum()

NameError: name 'predict_with_model' is not defined

In [59]:
(y_pred_c[:,0].astype(bool) & y_test.astype(bool)).sum()

6

In [61]:
y_pred = predict_with_model(X_test)
y_test = Y_test

from sklearn.metrics import precision_score, recall_score
y_pred_c = y_pred.round().astype(int)
precision = precision_score(y_test, y_pred_c)
recall = recall_score(y_test, y_pred_c)

loss, accuracy = model.evaluate(X_test, y_test)

print('Loss:', loss)
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
Y_test.sum(), y_pred_c.sum()

4/4 [==============================] - 1s 135ms/step - loss: 0.8821 - accuracy: 0.7398
Loss: 0.8820559978485107
Accuracy: 0.7398374080657959
Precision: 0.5454545454545454
Recall: 0.18181818181818182


(33.0, 11)